In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# 加载示例数据集
data = load_breast_cancer()
X, y = data.data, data.target

# 标准化特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 定义特征提取算法
feature_extractors = {
    "PCA": PCA(n_components=2),
    "SelectKBest": SelectKBest(f_classif, k=2),
    "LassoCV": LassoCV(),
    "RFE": RFE(estimator=LogisticRegression(), n_features_to_select=2),
    # 其他特征提取算法可以在此添加
}

# 提取特征并绘制图表
for name, extractor in feature_extractors.items():
    if name in ["LassoCV", "RFE"]:
        extractor.fit(X_scaled, y)
        if name == "LassoCV":
            feature_weights = extractor.coef_
        else:
            feature_weights = extractor.ranking_
    else:
        X_new = extractor.fit_transform(X_scaled, y)
        feature_weights = extractor.explained_variance_ratio_ if name == "PCA" else extractor.scores_

    plt.figure()
    plt.bar(range(len(feature_weights)), feature_weights)
    plt.title(f"Feature Weights using {name}")
    plt.xlabel("Feature Index")
    plt.ylabel("Weight")
    plt.show()

#########################模型优化#################################
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 定义模型和调优空间
models = {
    "LogisticRegression": LogisticRegression,
    "RandomForest": RandomForestClassifier,
    "SVM": SVC,
    "XGBoost": XGBClassifier,
    "LightGBM": LGBMClassifier,
    # 其他模型可以在此添加
}

def objective(trial, model_class):
    if model_class == LogisticRegression:
        params = {
            "C": trial.suggest_loguniform("C", 1e-5, 1e2),
            "max_iter": trial.suggest_int("max_iter", 100, 1000)
        }
    elif model_class == RandomForestClassifier:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 10, 200),
            "max_depth": trial.suggest_int("max_depth", 2, 32)
        }
    elif model_class == SVC:
        params = {
            "C": trial.suggest_loguniform("C", 1e-5, 1e2),
            "kernel": trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"]),
            "probability":trial.suggest_categorical("probability", [True])
        }
    elif model_class == XGBClassifier:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 10, 200),
            "max_depth": trial.suggest_int("max_depth", 2, 32),
            "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1.0)
        }
    elif model_class == LGBMClassifier:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 10, 200),
            "max_depth": trial.suggest_int("max_depth", 2, 32),
            "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1.0)
        }

    model = model_class(**params)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring=make_scorer(roc_auc_score)).mean()
    return score

optimized_models = {}

for name, model_class in models.items():
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, model_class), n_trials=50)
    optimized_models[name] = model_class(**study.best_params)

    print(f"Best parameters for {name}: {study.best_params}")



########################模型评估#############################
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, f1_score, average_precision_score

def bootstrap_ci(model, X_test, y_test, num_bootstrap=1000, alpha=0.95):
    metrics = {
        "AUC": [],
        "F1": [],
        "Precision": [],
        "Recall": [],
        "Accuracy": [],
        "Average Precision (AP)": []
    }
    n_size = len(y_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)
    
    for _ in range(num_bootstrap):
        indices = np.random.choice(range(n_size), size=n_size, replace=True)
        if len(np.unique(y_test[indices])) < 2:
            continue

        auc = roc_auc_score(y_test[indices], y_pred_proba[indices])
        f1 = f1_score(y_test[indices], y_pred[indices])
        precision = precision_score(y_test[indices], y_pred[indices])
        recall = recall_score(y_test[indices], y_pred[indices])
        accuracy = accuracy_score(y_test[indices], y_pred[indices])
        ap = average_precision_score(y_test[indices], y_pred_proba[indices])
        
        metrics["AUC"].append(auc)
        metrics["F1"].append(f1)
        metrics["Precision"].append(precision)
        metrics["Recall"].append(recall)
        metrics["Accuracy"].append(accuracy)
        metrics["Average Precision (AP)"].append(ap)
    
    ci_results = {}
    for metric, scores in metrics.items():
        lower = np.percentile(scores, ((1.0 - alpha) / 2.0) * 100)
        upper = np.percentile(scores, (alpha + ((1.0 - alpha) / 2.0)) * 100)
        ci_results[metric] = (np.mean(scores), (lower, upper))
    
    return ci_results

# 计算评估指标的置信区间
'''for name, model in optimized_models.items():
    try:
        model.fit(X_train, y_train)
        results_table = bootstrap_ci(model, X_test, y_test)
        print(f"{name} Classification CI Results:")
        print(results_table)
    except Exception as e:
        print(f"Error with model {name}: {e}")'''

results_dict = {}

for name, model in optimized_models.items():
    try:
        model.fit(X_train, y_train)
        results_table = bootstrap_ci(model, X_test, y_test)
        results_dict[name] = results_table
        print(f"{name} Classification CI Results:")
        print(results_table)
    except Exception as e:
        print(f"Error with model {name}: {e}")

print(results_dict)


##############################模型绘图###################################
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.calibration import calibration_curve
import numpy as np

def plot_curves(models, X_train, y_train, X_test, y_test, ci_results):
    colors = plt.cm.get_cmap('tab10', len(models))
    
    plt.figure(figsize=(20, 40))
    
    # ROC曲线 - 训练集
    plt.subplot(5, 2, 1)
    for i, (name, model) in enumerate(models.items()):
        model.fit(X_train, y_train)
        y_pred_proba_train = model.predict_proba(X_train)[:, 1]
        fpr_train, tpr_train, _ = roc_curve(y_train, y_pred_proba_train)
        plt.plot(fpr_train, tpr_train, label=f'{name} Train (AUC: {ci_results[name]["AUC"][0]:.2f} [{ci_results[name]["AUC"][1][0]:.2f}, {ci_results[name]["AUC"][1][1]:.2f}])', color=colors(i))
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve - Train')
    plt.legend()
    
    # ROC曲线 - 测试集
    plt.subplot(5, 2, 2)
    for i, (name, model) in enumerate(models.items()):
        model.fit(X_train, y_train)
        y_pred_proba_test = model.predict_proba(X_test)[:, 1]
        fpr_test, tpr_test, _ = roc_curve(y_test, y_pred_proba_test)
        plt.plot(fpr_test, tpr_test, label=f'{name} Test (AUC: {ci_results[name]["AUC"][0]:.2f} [{ci_results[name]["AUC"][1][0]:.2f}, {ci_results[name]["AUC"][1][1]:.2f}])', color=colors(i))
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve - Test')
    plt.legend()
    
    # PR曲线 - 训练集
    plt.subplot(5, 2, 3)
    for i, (name, model) in enumerate(models.items()):
        model.fit(X_train, y_train)
        y_pred_proba_train = model.predict_proba(X_train)[:, 1]
        precision_train, recall_train, _ = precision_recall_curve(y_train, y_pred_proba_train)
        plt.plot(recall_train, precision_train, label=f'{name} Train (AP: {ci_results[name]["Average Precision (AP)"][0]:.2f} [{ci_results[name]["Average Precision (AP)"][1][0]:.2f}, {ci_results[name]["Average Precision (AP)"][1][1]:.2f}])', color=colors(i))
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve - Train')
    plt.legend()
    
    # PR曲线 - 测试集
    plt.subplot(5, 2, 4)
    for i, (name, model) in enumerate(models.items()):
        model.fit(X_train, y_train)
        y_pred_proba_test = model.predict_proba(X_test)[:, 1]
        precision_test, recall_test, _ = precision_recall_curve(y_test, y_pred_proba_test)
        plt.plot(recall_test, precision_test, label=f'{name} Test (AP: {ci_results[name]["Average Precision (AP)"][0]:.2f} [{ci_results[name]["Average Precision (AP)"][1][0]:.2f}, {ci_results[name]["Average Precision (AP)"][1][1]:.2f}])', color=colors(i))
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve - Test')
    plt.legend()
    
    # 校准曲线 - 训练集hui
    plt.subplot(5, 2, 5)
    for i, (name, model) in enumerate(models.items()):
        model.fit(X_train, y_train)
        y_pred_proba_train = model.predict_proba(X_train)[:, 1]
        prob_true_train, prob_pred_train = calibration_curve(y_train, y_pred_proba_train, n_bins=10)
        plt.plot(prob_pred_train, prob_true_train, label=f'{name} Train', color=colors(i))
    plt.xlabel('Mean Predicted Probability')
    plt.ylabel('Fraction of Positives')
    plt.title('Calibration Curve - Train')
    plt.legend()
    
    # 校准曲线 - 测试集
    plt.subplot(5, 2, 6)
    for i, (name, model) in enumerate(models.items()):
        model.fit(X_train, y_train)
        y_pred_proba_test = model.predict_proba(X_test)[:, 1]
        prob_true_test, prob_pred_test = calibration_curve(y_test, y_pred_proba_test, n_bins=10)
        plt.plot(prob_pred_test, prob_true_test, label=f'{name} Test', color=colors(i))
    plt.xlabel('Mean Predicted Probability')
    plt.ylabel('Fraction of Positives')
    plt.title('Calibration Curve - Test')
    plt.legend()

    # 决策曲线分析 (DCA) - 训练集
    plt.subplot(5, 2, 7)
    thresholds = np.linspace(0.01, 0.99, 99)
    for i, (name, model) in enumerate(models.items()):
        model.fit(X_train, y_train)
        y_pred_proba_train = model.predict_proba(X_train)[:, 1]
        net_benefits_train = [net_benefit(y_pred_proba_train, y_train, threshold) for threshold in thresholds]
        plt.plot(thresholds, net_benefits_train, label=f'{name} Train', color=colors(i))
    treat_all_train = [np.sum(y_train) / len(y_train) - threshold / (1 - threshold) for threshold in thresholds]
    treat_none_train = [0 for _ in thresholds]
    plt.plot(thresholds, treat_all_train, linestyle='--', color='gray', label='Treat All Train')
    plt.plot(thresholds, treat_none_train, linestyle='--', color='black', label='Treat None Train')
    plt.xlabel('Threshold Probability')
    plt.ylabel('Net Benefit')
    plt.title('Decision Curve Analysis - Train')
    plt.legend()

    # 决策曲线分析 (DCA) - 测试集
    plt.subplot(5, 2, 8)
    for i, (name, model) in enumerate(models.items()):
        model.fit(X_train, y_train)
        y_pred_proba_test = model.predict_proba(X_test)[:, 1]
        net_benefits_test = [net_benefit(y_pred_proba_test, y_test, threshold) for threshold in thresholds]
        plt.plot(thresholds, net_benefits_test, label=f'{name} Test', color=colors(i))
    treat_all_test = [np.sum(y_test) / len(y_test) - threshold / (1 - threshold) for threshold in thresholds]
    treat_none_test = [0 for _ in thresholds]
    plt.plot(thresholds, treat_all_test, linestyle='--', color='gray', label='Treat All Test')
    plt.plot(thresholds, treat_none_test, linestyle='--', color='black', label='Treat None Test')
    plt.xlabel('Threshold Probability')
    plt.ylabel('Net Benefit')
    plt.title('Decision Curve Analysis - Test')
    plt.legend()

    # 模型预测概率分布 - 训练集
    plt.subplot(5, 2, 9)
    for i, (name, model) in enumerate(models.items()):
        model.fit(X_train, y_train)
        y_pred_proba_train = model.predict_proba(X_train)[:, 1]
        plt.hist(y_pred_proba_train, bins=20, alpha=0.5, label=f'{name} Train', color=colors(i))
    plt.xlabel('Predicted Probability')
    plt.ylabel('Frequency')
    plt.title('Predicted Probability Distribution - Train')
    plt.legend()

    # 模型预测概率分布 - 测试集
    plt.subplot(5, 2, 10)
    for i, (name, model) in enumerate(models.items()):
        model.fit(X_train, y_train)
        y_pred_proba_test = model.predict_proba(X_test)[:, 1]
        plt.hist(y_pred_proba_test, bins=20, alpha=0.5, label=f'{name} Test', color=colors(i))
    plt.xlabel('Predicted Probability')
    plt.ylabel('Frequency')
    plt.title('Predicted Probability Distribution - Test')
    plt.legend()

    plt.tight_layout()
    plt.show()

def net_benefit(probabilities, y_true, threshold):
        tp = np.sum((probabilities >= threshold) & (y_true == 1))
        fp = np.sum((probabilities >= threshold) & (y_true == 0))
        fn = np.sum((probabilities < threshold) & (y_true == 1))
        tn = np.sum((probabilities < threshold) & (y_true == 0))
        
        prevalence = (tp + fn) / (tp + fp + tn + fn)
        net_benefit = tp / (tp + fp + tn + fn) - (fp / (tp + fp + tn + fn)) * (threshold / (1 - threshold))
        return net_benefit


plot_curves(models=optimized_models,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test,ci_results=results_dict)


In [ ]:
def calculate_threshold(y_true, y_score, expected_metric="Youden", expected_value=0.5):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)

    if expected_metric == "Youden":
        j_scores = tpr - fpr
        j_ordered = sorted(zip(j_scores, thresholds), reverse=True)
        threshold = j_ordered[0][1]
    elif expected_metric.lower() == "sen" or expected_metric.lower() == "sensitivity":
        idx = np.where(tpr >= expected_value)[0]
        threshold = thresholds[idx[0]]
    else:
        raise ValueError("Not supported expected metric: {}".format(expected_metric))

    return threshold

calculate_threshold(y_true, y_score, expected_metric="sensitivity", expected_value=0.9)